In [ ]:
%pip install path_vis

In [ ]:
from path_vis import PathVis

PathVis(path=[
        [1, 0, 0, 0.0, 0, 1, 0, 0.0, 0, 0, 1, 0.0, 0, 0, 0, 1],
        [1, 0, 0, 0.2, 0, 1, 0, 0.1, 0, 0, 1, 0.0, 0, 0, 0, 1],
        [1, 0, 0, 0.4, 0, 1, 0, 0.15, 0, 0, 1, 0.1, 0, 0, 0, 1],
      ])

In [ ]:
import numpy as np

def rotz(phi):
    c, s = np.cos(phi), np.sin(phi)
    R = np.eye(4)
    R[:2, :2] = [[c, -s], [s, c]]
    return R

def roty(theta):
    c, s = np.cos(theta), np.sin(theta)
    R = np.eye(4)
    R[0, 0], R[0, 2] = c, s
    R[2, 0], R[2, 2] = -s, c
    return R

def segment_matrix(k, phi, s):
    if abs(k) < 1e-9:
        seg = np.eye(4)
        seg[2, 3] = s
    else:
        theta = k * s
        r = 1.0 / k
        seg = np.eye(4)
        seg[:3, :3] = roty(theta)[:3, :3]
        seg[0, 3] = r * (1 - np.cos(theta))
        seg[2, 3] = r * np.sin(theta)
        seg = rotz(phi) @ seg @ rotz(-phi)
    return seg

def arc_to_x(init_frame, kappa, l, phi, n):
    m = len(kappa)
    total_segments = m * n + 1
    curve = np.zeros((4, 4 * total_segments))
    curve[:, :4] = init_frame
    curr_frame = init_frame.copy()
    curr = 1

    for ki, li, pi in zip(kappa, l, phi):
        for j in range(1, n+1):
            seg = segment_matrix(ki, pi, j*li/n)
            curve[:, 4*curr:4*(curr+1)] = curr_frame @ seg
            curr += 1
        curr_frame = curr_frame @ segment_matrix(ki, pi, li)

    return curve

# Example usage
rng = np.random.default_rng()
kappa = rng.uniform(0,20,3)
l = np.round(rng.uniform(0.1,0.3,3),2)
phi = rng.uniform(0, 2*np.pi, 3)

init = np.eye(4)
curve = arc_to_x(init, kappa, l, phi, 10) # 10 segments per arc

# Flatten for PathVis
flat_curve = [curve[:, 4*i:4*(i+1)].flatten() for i in range(curve.shape[1]//4)]
flat_curve = np.ravel(flat_curve).tolist()

from path_vis import PathVis
PathVis(path=flat_curve)
